In [ ]:
using System;
using System.IO;
using System.Collections.Generic;
using System.Collections;
using System.Threading;
using System.Text.Json;

In [ ]:
var input = File.ReadAllLines("input.txt");

In [ ]:
public class Debug : IDisposable
{
    public static bool IsEnabled {get; private set;} = false;

    public static void Print(string input)
    {
        if(IsEnabled)
        {
            Console.WriteLine(input);
        }
    }

    public Debug(bool enable = true)
    {
        IsEnabled = enable;
    }

    public void Dispose()
    {
        IsEnabled = false;
    }
}

In [ ]:
public interface State
{
    bool IsGoal {get;}
    string Id {get;}
    IEnumerable<(State state, long cost)> Neighbours();
}

In [ ]:
public class Dijkstra
{
    public Dictionary<string, long> Cost = new Dictionary<string, long>();
    public PriorityQueue<State, long> Queue = new PriorityQueue<State, long>();

    public int Count = 0;
    
    public long Run(State start)
    {
        Queue.Enqueue(start, 0);

        State node;
        long cost;
        while(Queue.TryDequeue(out node, out cost))
        {            
            Count++;

            if(Count % 100000 == 0)
            {
                Debug.Print($"Visited {Count} nodes. {Queue.Count} left");
            }
            
            if(node.IsGoal)
            {
                break;
            }
            
            foreach(var n in node.Neighbours())
            {
                long total = n.cost + cost;

                if(Cost.TryGetValue(n.state.Id, out var c))
                {
                    if(c <= total)
                    {
                        continue;
                    }
                }
                Cost[n.state.Id] = total;
                Queue.Enqueue(n.state, total);
            }
        }
        Debug.Print("Done?");

        if(node?.IsGoal == true)
        {
            Debug.Print("YES!");
            return cost;
        }
        else{
            Debug.Print("NO!");
            return -1;
        }
    }
}

In [ ]:
public static class Graph
{
    public static List<string> N {get; set;} = new List<string>();
    public static Dictionary<(string from, string to),long> H {get; set;} = new Dictionary<(string, string), long>();
    public static void Load(IEnumerable<string> input)
    {
        foreach(var line in input)
        {
            var seg = line.Split(new []{' ', '.'}, 11, StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries);
            H[(seg[0], seg[10].Trim('.'))] = long.Parse(seg[3]) * (seg[2] == "gain" ? 1 : -1);

            if(!N.Contains(seg[0]))
            {
                N.Add(seg[0]);
            }
        }
    }
}

In [ ]:
public class TablePlan : State
{
    public List<string> Plan {get; init;}
    public bool IsGoal => Plan?.Count() == Graph.N.Count();
    public string Id => Plan == null ? "empty" : string.Join(';', Plan);
    public IEnumerable<(State state, long cost)> Neighbours()
    {
        if(Plan == null)
        {
            yield return (new TablePlan{Plan = new List<string>{Graph.N.First()}}, 0);
            yield break;
        }

        foreach(var to in Graph.N.Except(Plan))
        {
            var from = Plan.Last();

            if(Plan.Count() == Graph.N.Count() - 1)
            {
                var first = Plan.First();
                yield return (new TablePlan{Plan = Plan.Append(to).ToList()}, -(Graph.H[(from, to)] + Graph.H[(to, from)] + Graph.H[(first, to)] + Graph.H[(to, first)]));
            }
            else{
                yield return (new TablePlan{Plan = Plan.Append(to).ToList()}, -(Graph.H[(from, to)] + Graph.H[(to, from)]));
            }
        }
    }
}

In [ ]:
Graph.Load(input);

In [ ]:
// Graph.H.Display();
// Graph.N

In [ ]:
var d = new Dijkstra();
using(new Debug())
{
    d.Run(new TablePlan()).Display();
}

Start
First
Alice
Start
Continue
Bob
Continue
Carol
Continue
David
Start
Continue
Carol
Continue
David
Start
Continue
David
Done?
YES!


-330